# TTA (Test Time Augmentation)

In [1]:
from modules.utils import load_yaml, rle_decode, rle_encode, draw_fig
from modules.optimizer import get_optimizer
from modules.scheduler import get_scheduler
from modules.loss import get_loss_function
from modules.model import get_transformers_model, get_smp_model
from modules.dataset import transformsCustomDataset, smpDataset
from modules.augmentation import *

import os
import random
import shutil
import argparse
from glob import glob
from tqdm import tqdm
from datetime import datetime

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from transformers import (
    AutoImageProcessor,
    TrainingArguments, 
    Trainer,
    SegformerConfig
)

/home/vip/anaconda3/envs/seg/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prj_dir = './'
config_path = os.path.join(prj_dir, 'config', 'predict_seg_512.yaml')
config = load_yaml(config_path)

In [3]:
torch.cuda.manual_seed(config['seed'])
torch.manual_seed(config['seed'])
np.random.seed(config['seed'])
random.seed(config['seed'])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
df = pd.read_csv(f"data/test.csv")

image_paths = []
for img in df["img_path"]:
    path = f"./data/test_img/{os.path.basename(img)}"
    image_paths.append(path)

In [5]:
model = get_transformers_model(name=config['model']['name'])
model = model.from_pretrained(
    config['model']['weight'],
    num_labels=1,
    ignore_mismatched_sizes=True
).to(device)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.2.proj.weight', 'decode_head.classifier.weight', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.linear_fuse.weight', 'decode_head.linear_c.0.proj.weight', 'decode_head.batch_norm.bias', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.batch_norm.running_mean', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.batch_norm.running_var']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
weights = torch.load('./results/train/20230717_135532/weights/best.pth')
# model.load_state_dict(weights['model'])
model.load_state_dict(weights)

<All keys matched successfully>

In [7]:
class TTA:
    def __init__(self, transform):

        self.processor = AutoImageProcessor.from_pretrained(config['processor'])

        self.transform = transform

        self.transforms = [
            A.VerticalFlip(p=1),
            A.HorizontalFlip(p=1),
            A.Compose([
                A.HorizontalFlip(p=1),
                A.VerticalFlip(p=1),
            ]),
            A.Rotate(limit=[90, 90], p=1),
            A.Rotate(limit=[-90, -90], p=1),
        ]

    def process(self, image):
        images = []

        image = self.transform(image=image)['image']
        inputs = self.processor(image, return_tensors='pt')
        inputs = {k:v.squeeze(0) for k, v in inputs.items()}
        images.append(inputs['pixel_values'])

        for t in self.transforms:
            img = self.transform(image=t(image=image)['image'])['image']

            inputs = self.processor(img, return_tensors='pt')
            inputs = {k:v.squeeze(0) for k, v in inputs.items()}

            images.append(inputs['pixel_values'])

        # return images
        return torch.tensor(np.array(images), dtype=torch.float32)

    def unprocess(self, images):
        results = []

        results.append(images[0])
        results.append(self.transforms[0](image=images[1])['image'])
        results.append(self.transforms[1](image=images[2])['image'])
        results.append(self.transforms[2](image=images[3])['image'])
        results.append(self.transforms[4](image=images[4])['image'])
        results.append(self.transforms[3](image=images[5])['image'])

        return results

transform = A.Compose([
    # A.Normalize(),
    # ToTensorV2(),
])

tta = TTA(transform=transform)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/home/vip/anaconda3/envs/seg/lib/python3.11/site-packages/transformers/models/segformer/image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [8]:
processor = AutoImageProcessor.from_pretrained(config['processor'])

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [9]:
def upscale_logits(logit_outputs, res=224):
    return nn.functional.interpolate(
        logit_outputs,
        size=(res, res),
        mode='bilinear',
        align_corners=False
    )

In [15]:
result = []

with torch.no_grad():
    model.eval()
    for img_path in tqdm(image_paths):
        image = cv2.imread(img_path)
        images = tta.process(image)
        images = images.to(device)

        outputs = model(images)
        predicts = upscale_logits(outputs.logits)

        seg_prob = torch.sigmoid(predicts).detach().cpu().numpy().squeeze()
        seg = (seg_prob > 0.5).astype(np.uint8)

        tta_seg = tta.unprocess(seg)

        tta_prob = (tta_seg[0] + tta_seg[1] + tta_seg[2] + tta_seg[3] + tta_seg[4] + tta_seg[5])
        tta_image = (tta_prob >= 3.0).astype(np.uint8)

        mask_rle = rle_encode(tta_image)
        if mask_rle == '':
            result.append(-1)
        else:
            result.append(mask_rle)

100%|██████████| 60640/60640 [1:52:44<00:00,  8.96it/s]  


# Submission

In [16]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['mask_rle'] = result

In [17]:
submit.to_csv('./submit13.csv', index=False)

In [18]:
from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
    './submit13.csv', 
    '5eca29221f8e6e442f5d55ccb7455756c26e5f85a5d1aac8208a97db790bbdb9', 
    '236092', 
    'ADED', 
    ''
)

{'isSubmitted': True, 'detail': 'Success'}
